# Data

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:

from backbone.utils.wfo_utils import run_strategy, run_wfo
from backbone.utils.general_purpose import calculate_units_size, diff_pips

import itertools
import numpy as np
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
from pandas import Timestamp
import numpy as np
import random
from backtesting import Strategy
from backtesting.lib import crossover
import talib as ta
import yfinance as yf

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

random.seed(42)

In [3]:
INITIAL_CASH = 10_000
MARGIN = 1/30
COMMISSION = 7e-4

timezone = pytz.timezone("Etc/UTC")
date_from_get_data = datetime(2021, 10, 1, tzinfo=timezone)
date_to_get_data = datetime(2024, 9, 1, tzinfo=timezone)

#
limited_testing_start_date = Timestamp('2022-09-01 00:00:00', tz='UTC')
limited_testing_end_date = Timestamp('2022-12-01 00:00:00', tz='UTC')

In [4]:

groups = [
    'Forex_Indicator',
    'CryptoCross_grp',
    'Crypto_group',
    'Energies_group',
    'Forex_group',
    'Indices_group',
    'Stocks_group'
]

In [5]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

tickers = ['US500m','USTECm']

print(tickers)

intervals = [
    mt5.TIMEFRAME_H1,
]

symbols = get_data(tickers, intervals, date_from_get_data, date_to_get_data)

['US500m', 'USTECm']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
US500m
USTECm


In [6]:
vix = yf.Ticker("^VIX").history(interval='1h', period='1y')
vix.rename(
    columns={'Close':'VixClose'}, inplace=True
)
vix.index = vix.index.tz_convert('UTC')

vix

,Open,High,Low,VixClose,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-11-01 07:00:00+00:00,18.020000,18.090000,17.980000,18.090000,0,0.0,0.0
2023-11-01 08:00:00+00:00,18.030001,18.340000,17.980000,18.309999,0,0.0,0.0
2023-11-01 09:00:00+00:00,18.250000,18.370001,18.180000,18.340000,0,0.0,0.0
2023-11-01 10:00:00+00:00,18.299999,18.379999,18.209999,18.250000,0,0.0,0.0
2023-11-01 11:00:00+00:00,18.309999,18.420000,18.209999,18.360001,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-10-31 16:00:00+00:00,22.959999,23.040001,22.330000,22.410000,0,0.0,0.0
2024-10-31 17:00:00+00:00,22.410000,22.900000,22.180000,22.860001,0,0.0,0.0
2024-10-31 18:00:00+00:00,22.860001,23.260000,22.629999,22.820000,0,0.0,0.0


In [7]:
max_start_date = None
intervals_start_dates = {}

tickers = np.unique(list(symbols.keys())).tolist()

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date


intervals_start_dates

print(intervals_start_dates)
print(limited_testing_start_date)
print(limited_testing_end_date)

{16385: Timestamp('2021-10-27 17:00:00+0000', tz='UTC')}
2022-09-01 00:00:00+00:00
2022-12-01 00:00:00+00:00


# Estrategia

In [8]:
from backbone.trader_bot import TraderBot

def ll_hh_indicator(close, window=None):
    if type(close) != pd.Series:
        close = close.s

    rolling_min = close.rolling(window=window, min_periods=1).min()
    is_lower_low = close == rolling_min

    rolling_max = close.rolling(window=window, min_periods=1).max()
    is_higher_high = close == rolling_max

    return is_lower_low, is_higher_high

class VixRsi(Strategy):
    pip_value = None
    minimum_units = None
    maximum_units = None
    contract_volume = None
    opt_params = None
    
    vix_percentage_above_sma = 0.05
    ll_hh_window = 5
    rsi_threshold = 50
    sma_period = 200
    vix_sma_period = 10
    rsi_period = 3
    risk=1
    atr_multiplier = 1.5
    
    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, timeperiod=self.rsi_period)
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=self.sma_period)
        self.vix_sma = self.I(ta.SMA, self.data.VixClose, timeperiod=self.vix_sma_period)
        self.lower_low, self.higher_high = self.I(ll_hh_indicator, self.data.Close, window=self.ll_hh_window)
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)
        

    def next(self):
        actual_close = self.data.Close[-1]
        cum_rsi = self.rsi[-1] + self.rsi[-2]
        vix_sma_value = self.vix_sma[-1]
        vix_close = self.data.VixClose[-1]
        vix_above_sma = vix_close > (vix_sma_value * (1 + self.vix_percentage_above_sma))

        if self.position:
            first_trade = self.trades[0]
            today = self.data.index[-1].tz_convert('UTC')
            time_in_position = (today - first_trade.entry_time.tz_convert('UTC'))
            time_in_position = time_in_position.days

            if self.position.is_long:
                if not vix_above_sma:
                    self.position.close()

        else:

            if vix_above_sma and cum_rsi <= self.rsi_threshold and actual_close > self.sma[-1]:
            # if vix_above_sma and actual_close > self.sma[-1]:
                sl_price = self.sma[-1] - self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value, 
                    maximum_units=self.maximum_units, 
                    minimum_units=self.minimum_units
                )
                
                self.buy(
                    size=units,
                    # sl=sl_price
                )
                
    def next_live(self, trader:TraderBot):
        actual_close = self.data.Close[-1]

        positions = trader.get_open_positions()
        if positions and positions[-1].type == mt5.ORDER_TYPE_BUY:
            if self.higher_high:
                trader.close_order(positions[-1])

        else:
            cum_rsi = self.rsi[-1] + self.rsi[-2]

            vix_sma_value = self.vix_sma[-1]
            vix_close = self.data.VixClose[-1]
            vix_above_sma = vix_close > (vix_sma_value * (1 + self.vix_percentage_above_sma))

            if vix_above_sma and cum_rsi <= self.rsi_threshold and actual_close > self.sma[-1]:
                info_tick = trader.get_info_tick()
                price = info_tick.ask
                
                capital_to_risk = trader.equity * self.risk / 100
                units = capital_to_risk / price
                
                lots = round(units / trader.contract_volume, 2)
                
                trader.open_order(
                    type_='buy',
                    price=price,
                    size=lots
                )

STRATEGY = VixRsi

# Analisis preliminar

In [9]:
strategies = [
    STRATEGY
]

experiments = list(itertools.product(
    tickers, intervals, strategies
))

stats_per_symbol = {}

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:
    try:
        print(ticker, interval)
        
        if ticker not in stats_per_symbol.keys():
            stats_per_symbol[ticker] = {}
        
        prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date].copy()
        
        prices.index = prices.index.tz_convert('UTC')
        merge = pd.merge(
            prices, 
            vix[['VixClose']], 
            left_index=True, 
            right_index=True
        )
        
        df_stats, stats = run_strategy(
            strategy=strategy,
            ticker=ticker,
            interval=interval,
            commission=COMMISSION, 
            prices=merge, 
            initial_cash=INITIAL_CASH, 
            margin=MARGIN
        )

        performance = pd.concat([performance, df_stats])
        stats_per_symbol[ticker][interval] = stats

    except Exception as e:
        print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)

US500m 16385
hubo un problema con US500m 16385: OHLC `data` is empty
USTECm 16385
hubo un problema con USTECm 16385: OHLC `data` is empty


KeyError: 'return'

In [ ]:
filter_performance = performance.sort_values(by=['return/dd'], ascending=[False])#.drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

In [ ]:
for ticker, interval in zip(portfolio, intervals):
    
    prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date].copy()
    
    prices.index = prices.index.tz_convert('UTC')
    merge = pd.merge(
        prices, 
        vix[['VixClose']], 
        left_index=True, 
        right_index=True
    )

    df_stats = run_strategy(
        strategy=STRATEGY,
        ticker=ticker,
        interval=interval,
        commission=COMMISSION, 
        prices=merge, 
        initial_cash=INITIAL_CASH, 
        margin=MARGIN,
        plot=True
    )


In [ ]:
0/0

# WFO

In [ ]:
from backbone.utils.wfo_utils import optimization_function


lookback_bars = 1800
validation_bars = 180
warmup_bars = 50

all_wfo_performances = pd.DataFrame()
all_opt_params = {}

params = {
    'vix_percentage_above_sma': list(np.arange(0.01, 0.2, 0.01).round(2)),
    'sma_period': [50],
}

for index, row in filter_performance.iterrows():
    
    try:
        ticker = row.ticker
        interval = row.interval
        
        print(ticker, interval)
        
        prices = symbols[ticker][interval].copy()
        
        prices.index = prices.index.tz_convert('UTC')
        merge = pd.merge(
            prices, 
            vix[['VixClose']], 
            left_index=True, 
            right_index=True
        )

        wfo_stats, df_stats, opt_params = run_wfo(
            strategy=STRATEGY,
            ticker=ticker,
            interval=interval,
            prices=merge,
            initial_cash=INITIAL_CASH,
            commission=COMMISSION,
            margin=MARGIN,
            optim_func=optimization_function,
            params=params,
            lookback_bars=lookback_bars,
            warmup_bars=warmup_bars,
            validation_bars=validation_bars,
            plot=True
        )
                
        if ticker not in all_opt_params.keys():
            all_opt_params[ticker] = {}

        all_opt_params[ticker][interval] = opt_params

        all_wfo_performances = pd.concat([all_wfo_performances, df_stats])
    
    except Exception as e:
        print(f'No se pudo ejecutar para el ticker {ticker}: {e}')
    
all_wfo_performances['return/dd'] = all_wfo_performances['return'] / -all_wfo_performances['drawdown']
all_wfo_performances['drawdown'] = -all_wfo_performances['drawdown']
all_wfo_performances['custom_metric'] = (all_wfo_performances['return'] / (1 + all_wfo_performances.drawdown)) * np.log(1 + all_wfo_performances.trades)

all_wfo_performances.drawdown_duration = pd.to_timedelta(all_wfo_performances.drawdown_duration)
all_wfo_performances.drawdown_duration = all_wfo_performances.drawdown_duration.dt.days

In [ ]:

performance = pd.DataFrame()

wfo_stats_per_symbol = {}

for index, row in filter_performance.iterrows():
    try:
        ticker = row.ticker
        interval = row.interval
        print(ticker, interval)
        
        params = all_opt_params[ticker][interval]
        
        prices = symbols[ticker][interval].copy()
        prices.index = prices.index.tz_convert('UTC')
        merge = pd.merge(
            prices, 
            vix[['VixClose']], 
            left_index=True, 
            right_index=True
        )
        
        
        if ticker not in wfo_stats_per_symbol.keys():
            wfo_stats_per_symbol[ticker] = {}

        df_stats, wfo_stats = run_strategy(
            strategy=strategy,
            ticker=ticker,
            interval=interval,
            commission=COMMISSION, 
            prices=merge, 
            initial_cash=INITIAL_CASH, 
            margin=MARGIN, 
            opt_params=params, 
            plot=True
        )

        wfo_stats_per_symbol[ticker][interval] = wfo_stats

        performance = pd.concat([performance, df_stats])

    except Exception as e:
        print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


In [ ]:
filtered_wfo_performance = performance[
    (performance['stability_ratio'] > 0.7)
].sort_values(by='return/dd', ascending=False)

filtered_wfo_performance

# Montecarlo

In [15]:
from backbone.utils.montecarlo_utils import montecarlo_statistics_simulation

# Crear una lista para almacenar los resultados de cada ticker
data_drawdown = []
data_return = []
montecarlo_simulations = {}

all_drawdowns = pd.DataFrame()
all_returns = pd.DataFrame()

for index, row in filtered_wfo_performance.iterrows():
    ticker = row.ticker
    interval = row.interval
    
    print(f"Procesando ticker: {ticker}")
    trades_history = stats_per_symbol[ticker][interval]._trades
    eq_curve = stats_per_symbol[ticker][interval]._equity_curve
    
    # Simulación de Montecarlo para cada ticker (datos agregados)
    mc, synthetic_drawdown_curve, synthetic_return_curve = montecarlo_statistics_simulation(
        equity_curve=eq_curve,
        trade_history=trades_history, 
        n_simulations=100_000, 
        initial_equity=INITIAL_CASH, 
        threshold_ruin=0.8, 
        return_raw_curves=True,
        percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
    )
    
    montecarlo_simulations[ticker] = mc
    
    synthetic_drawdown_curve['ticker'] = ticker
    synthetic_return_curve['ticker'] = ticker
    
    all_drawdowns = pd.concat([all_drawdowns, synthetic_drawdown_curve])
    all_returns = pd.concat([all_returns, synthetic_return_curve])

In [ ]:
dd_df = pd.DataFrame()
returns_df = pd.DataFrame()

for ticker, mc in montecarlo_simulations.items():
    mc = mc.rename(
        columns={
            'Drawdown (%)': f'drawdown_{ticker}',
            'Final Return (%)': f'return_{ticker}',
        }
    )

    if dd_df.empty:
        dd_df = mc[[f'drawdown_{ticker}']]
    
    else:
        dd_df = pd.merge(
            dd_df,
            mc[[f'drawdown_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
    if returns_df.empty:
        returns_df = mc[[f'return_{ticker}']]
    
    else:
        returns_df = pd.merge(
            returns_df,
            mc[[f'return_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
display(dd_df)
display(returns_df)


In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(25, 18))
sns.boxplot(data=all_drawdowns, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = all_drawdowns["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = all_drawdowns["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()


In [ ]:
# Generar el gráfico
plt.figure(figsize=(12, 20))
sns.boxplot(data=all_returns, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = all_returns["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = all_returns["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()
